In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
import pickle
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
np.random.seed(1234)

Using TensorFlow backend.


In [2]:
def build_model(layers, X_variables):
        
    # Initialise model
    model = Sequential()
    
    # Add the first lSTM layer
    model.add(LSTM(
            layers[1],
            input_shape=(None, 5),
            return_sequences=True))
    model.add(Dropout(0.2))
    
    # Add the second lSTM layer 
    model.add(LSTM(
            layers[2],
            return_sequences=False))
    model.add(Dropout(0.2))
    
    # The last layer we use is a Dense layer ( = feedforward). 
    # Since we are doing a regression, its activation is linear.
    model.add(Dense(
            layers[3]))
    model.add(Activation("linear"))
    
    # Lastly, we compile the model using a Mean Square Error 
    # (again, it's standard for regression) and the RMSprop optimizer. 
    # See the mnist example to learn more on rmsprop.
    start = time.time()
    model.compile(loss="mse", optimizer="rmsprop")
    print("Compilation Time : ", time.time() - start)
    return model

In [3]:
def run_network(model, data, sequence_length):
    epochs = 1
    ratio = 0.5
    start_time = time.time()
    
    X_train, y_train, X_test, y_test = data
    
    print('\nData Loaded. Compiling...\n')
        
    try:
        model.fit(
            X_train, y_train,
            batch_size=512, epochs=epochs, validation_split=0.05)
        predicted = model.predict(X_test)
        predicted = np.reshape(predicted, (predicted.size,))
    except KeyboardInterrupt:
        print('Training duration (s) : ', time.time() - global_start_time)
        return model, y_test, 0
    
    try:
        fig = plt.figure()
        ax = fig.add_subplot()
        ax.plot(y_test[:100, 0])
        plt.plot(predicted[:100, 0])
        plt.show()
    except Exception as e:
        print('Error: ' + str(e))
    print('Training duration (s) : ', time.time() - start_time)
    return model, X_test, y_test, predicted

The only parameters you should need to change

In [4]:
# path to pickled file of model ready dataset: data = [X_train, y_train, X_test, y_test]
path_to_dataset = 'D:/bitfinexUSD_ready.pk'
# Length of sliding window
sequence_length = 50
# Network has a 5D input, two hidden layers of size 50, 100 and a 1D output layer
layers = [5, 50, 100, 1] 
# Number of variables per timestep
X_variables = 5  # OHLC, volume

In [5]:
with open(path_to_dataset, 'rb') as handle:
    [X_train, y_train, X_test, y_test] = pickle.load(handle)

In [6]:
print(X_train.shape)  # training sequences, sliding window length, number of input variables (ohlc  volume)
print(y_train.shape)  # training sequences, predicted variable (the close of the next time period)
print(X_test.shape)  # test sequences, sliding window length, number of input variables (ohlc  volume)
print(y_test.shape)  # test sequences, predicted variable (the close of the next time period)

(449955, 49, 5)
(449955,)
(49995, 49, 5)
(49995,)


In [7]:
model = build_model(layers, X_variables)

Compilation Time :  0.05338621139526367


In [8]:
model, X_test, y_test, predicted = \
    run_network(model, [X_train, y_train, X_test, y_test], sequence_length)


Data Loaded. Compiling...

Train on 427457 samples, validate on 22498 samples
Epoch 1/1
427457/427457 [==============================] - 625s 1ms/step - loss: 37903.0270 - val_loss: 26968.4725
Error: 'NoneType' object has no attribute 'plot'
Training duration (s) :  664.4156634807587


In [9]:
previous_actual_close = X_test[:,-1,-2]
actual_close = y_test
predicted_close = predicted

In [10]:
assert(previous_actual_close.shape == actual_close.shape == predicted_close.shape)
print('    t0            t1           t2           t3')
print(previous_actual_close[:3])
print('            ', actual_close[:3])
print('            ', predicted_close[:3])

    t0            t1           t2           t3
[-82.34185791 -81.86184692 -81.87185669]
             [-81.86184692 -81.87185669 -81.90185547]
             [-88.70996094 -88.7663269  -88.76239777]
